### Coursework 2

In this coursework you will be aiming to complete two classification tasks. 
Both the classification tasks relate to text classification tasks. 

One task is to be solved using Support Vector Machines. The other has to be solved using Boosting.

The specific tasks and the marking for the various tasks are provided in the notebook. Each task is expected to be accompanied by a lab-report. Each task can have a concise lab report that is maximum of one page in an A4 size. You will be expected to submit your Jupyter Notebook and all lab reports as a single zip file. You could have additional functions implemented that you require for carrying out each task.


#### Task 1

In this task, you need to obtain sentiment analysis for the provided dataset. The dataset consists of movie reviews with the sentiments being provided. The sentiments are either positive or negative. You need to train an SVM based classifier to obtain train and check on the sample test dataset provided. The method will be evaluated also against an external test set. Please do not hardcode any dimensions or number of samples while writing the code. It should be possible to automate the testing and hardcoding values does not allow for automated testing. 

You are allowed to use scikit-learn to implement the SVM. However, you are expected to write your own kernels.

You are allowed to use the existing library functions such as scikit-learn or numpy for obtaining the SVM. The main idea is to analyse the dataset using different kind of kernels. You are also supposed to write your own custom text kernels. Refer to the documentation provided [here](https://scikit-learn.org/stable/modules/svm.html) at 1.4.6.2 and an example [here](https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html) for writing your own kernels.

Details regarding the marking have been provided in the coursework specification file. Ensure that the code can be run with different test files. 

#### Process the text and obtain a bag of words-based features 

In [2]:

import numpy as np
import nltk
import pandas as pd


def load_data(train_file, test_file):
    # Read the CSV file and extract Bag of Words Features
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    
    raw_X_train = list(train_df.review)
    y_train = list(map( (lambda x : 0 if x == 'negative' else 1), train_df.sentiment))
    
    raw_X_test = list(test_df.review)
    y_test = list(map( (lambda x : 0 if x == 'negative' else 1), test_df.sentiment))
    
    return raw_train_x, train_y, raw_test_x, test_y 

def extract_bag_of_words_train_test(train_file, test_file):
    
    # Read the CSV file and extract Bag of Words Features
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    
    raw_X_train = list(train_df.review)
    y_train = list(map( (lambda x : 0 if x == 'negative' else 1), train_df.sentiment))
    
    raw_X_test = list(test_df.review)
    y_test = list(map( (lambda x : 0 if x == 'negative' else 1), test_df.sentiment))
    
    X_train = get_embeddings(raw_X_train)
    X_test  = get_embeddings(raw_X_test)
    return (X_train, y_train, X_test, y_test)

# def install_embedding_model():
#     import importlib.util
#     import subprocess
#     import sys
   
#     package_name = 'sentence-transformers'

#     #check if already installed
#     spec = importlib.util.find_spec(package_name)
#     if spec is None:
#         subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

def get_embeddings(reviews):
    #
    from sentence_transformers import SentenceTransformer

    model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')#'sentence-transformers/paraphrase-MiniLM-L3-v2')
    embeddings = model.encode(reviews)
    return embeddings


train_file = 'movie_review_test.csv'
test_file =  'movie_review_test.csv'

#x_test, y_train, x_test, y_test = extract_bag_of_words_train_test(train_file, test_file)
#embeddings = get_embeddings(x)

# print(len(x), len(y))
# print(len(embeddings))

In [ ]:
install_embedding_model()

(769,)


In [9]:


class SVMClassifier:
    def __init__(self):
        import numpy as np
        from sklearn import svm
        #implement initialisation
        #self.some_paramter=1
        self.clf = svm.SVC(decision_function_shape='ovo')
        self.clf.kernel = 'rbf'
        
    # define your own kernel here
    # Refer to the documentation here: https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html
    def fit(self, X,y):
        # training of the SVM
        # Ensure you call your own defined kernel here

        self.clf.fit(X, y)
               
        return
    
    def predict(self, X):
        # prediction routine for the SVM
        predictions = self.clf.predict(X)
        #print('predictions:', len(predictions))
        
        return predictions

### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function. 

In [87]:
from sklearn.metrics import accuracy_score

def test_func_svm(dataset_train, dataset_test):    
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    sc = SVMClassifier()
    #print(X_train, Y_train)
    sc.fit(X_train, Y_train)
    Y_Pred = sc.predict(X_test)
    acc = accuracy_score(Y_test, Y_Pred)
    print("Accuracy:",acc)
    return acc

def test_svm(model, train_x, train_y, test_x, test_y):  
    sc = model 
    #print(X_train, Y_train)
    sc.fit(train_x, train_y)
    Y_Pred = sc.predict(test_x)
    acc = accuracy_score(test_y, Y_Pred)
    #print("Accuracy:",acc)
    return acc

def crossval_svm(model, x, y, kfolds = 5):
    from sklearn.model_selection import KFold
    
    kfold_accuracys = []
    
    kf = KFold(n_splits = kfolds)
    train_x = kf.split(x,y)
    #print(train_x)
    
    
    for train_index, val_index in kf.split(x):
        train_x = x[train_index]
        train_y = y[train_index]
        val_x = x[val_index]
        val_y = y[val_index]
        
        acc = test_svm(model, train_x, train_y, val_x, val_y)
        #print(acc)
        kfold_accuracys.append(acc)
        
    return np.mean(kfold_accuracys)
        


In [88]:
#X_train, y_train, X_test, y_test  = extract_bag_of_words_train_test("movie_review_train.csv", "movie_review_test.csv")

model = SVMClassifier()
cross_val_acc = crossval_svm(model, X_train, np.array(y_train))

test_acc = test_func_svm("movie_review_train.csv", "movie_review_test.csv")

print("Cross val accuracy: ", cross_val_acc, "Test accuracy: ", test_acc)

0.81
0.808
0.793
0.797
0.812
Accuracy: 0.798
Cross val accuracy:  0.804 Test accuracy:  0.798


In [ ]:
### misclassification hyperparam tuning

for i in range(10):
    model = SVMClassifier()
    model.kernel = 

### Task 2

In this task you need to implement a boosting based classifier that can be used to classify the images. 

Details regarding the marking for the coursework are provided in the coursework specification file. Please ensure that your code will work with a different test file than the one provided with the coursework.

Note that the boosting classifier you implement can include decision trees from scikit-learn or your own decision trees. Use the same sentiment analysis dataset for evaluation.

In [4]:
class BoostingClassifier:
    # You need to implement this classifier. 
    def __init__(self):
        import numpy as np
        #implement initialisation
        self.some_paramter=1
    def fit(self, X,y):
        from sklearn.tree import DecisionTreeClassifier
        import numpy as np
        #implement training of the boosting classifier
        return 
    def predict(self, X):
        # implement prediction of the boosting classifier
        return

### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function. 

In [5]:
def test_func_boosting(dataset_train, dataset_test):
    from sklearn.metrics import accuracy_score    
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    bc = BoostingClassifier()
    bc.fit(X_train, Y_train)
    Y_Pred = bc.predict(X_test)    
    acc = accuracy_score(Y_test, Y_Pred)
    return acc

In [ ]:
acc = test_func_boosting("movie_review_train.csv", "movie_review_test.csv")